In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [7]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# List Parser

In [3]:
# Additional imports for list parser.
from langchain.output_parsers import CommaSeparatedListOutputParser

In [9]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0.9)

# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()

print()

# Define a query as a string, combining with the instructions.
query = "List 3 Spiderman comics" + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)


Amazing Spider-Man, Ultimate Spider-Man, Spider-Verse

['Amazing Spider-Man', 'Ultimate Spider-Man', 'Spider-Verse']


# Structured Parser

In [10]:
# Additional imports for structured parser.
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [13]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0.9)

# Define the schemas for our parser.
schemas = [
    ResponseSchema(name='country', description='the country'),
    ResponseSchema(name='capital', description='the capital')
]

# Initialize the output parser using the schema.
parser = StructuredOutputParser.from_response_schemas(schemas)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()

print()

# Define a query as a string, combining with the instructions.
query = "Name a country and its capital." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)
print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print(data['country'])
print(data['capital'])
print()

# Define a new query using the parsed output.
query2 = f"What are three tourist attractions in {data['capital']}?"

# Pass the query to the invoke method, and print the result.
result2 = llm.invoke(query2)
print(result2.content)


```json
{
	"country": "France",
	"capital": "Paris"
}
```

{'country': 'France', 'capital': 'Paris'}
France
Paris

1. Eiffel Tower: One of the most iconic landmarks in the world, the Eiffel Tower offers stunning views of Paris from its observation decks. Visitors can also dine at the tower's restaurant or enjoy a picnic in the surrounding Champ de Mars park.

2. Louvre Museum: Home to thousands of works of art, including the famous Mona Lisa, the Louvre is one of the largest and most visited museums in the world. Visitors can explore art and artifacts from various historical periods and cultures in the museum's impressive galleries.

3. Notre-Dame Cathedral: This historic cathedral is a masterpiece of French Gothic architecture and a symbol of Paris. Visitors can admire the intricate stone carvings, stunning stained glass windows, and climb to the top for panoramic views of the city. Despite the fire in 2019, Notre-Dame remains a must-see attraction in Paris.


# Class-based Parser

In [19]:
# Additional imports for Pydantic parser.
from langchain.output_parsers import PydanticOutputParser
from pydantic import Field, BaseModel

In [25]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0.9)

# Define the class for our parsed responses.
class Country(BaseModel): 
    name: str = Field(description="The country name")
    capital: str = Field(description="The capital city")
    population: int = Field(description="the country's population")

# Initialize the output parser using the schema.
parser = PydanticOutputParser(pydantic_object=Country)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()
print(instructions)
print()

# Define a query as a string, combining with the instructions.
query = "Name a country, its capital and its total population" + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)
print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print("Country: ", data.name)
print("Capital: ", data.capital)
print("Population: ", data.population)
print()

# Define a new query using the parsed output.
query2 = f"What are 3 tourist attractions in {data.capital}"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query2)
print(result.content)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "The country name", "title": "Name", "type": "string"}, "capital": {"description": "The capital city", "title": "Capital", "type": "string"}, "population": {"description": "the country's population", "title": "Population", "type": "integer"}}, "required": ["name", "capital", "population"]}
```

{
  "name": "Japan",
  "capital": "Tokyo",
  "population": 126476461
}

name='Japan' capital='Tokyo' population=126476461
Country:  Japan
Capital:  Tokyo
Population:  126476461

1. Tokyo Disneyland: A popular them